In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

from tensorflow.keras.utils import normalize

x_train = normalize(x_train)
x_test = normalize(x_test)

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0.0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        brightness_range=None,  
        shear_range=0.0,  # set range for random shear
        zoom_range=0.0,  # set range for random zoom
        channel_shift_range=0.0,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.0,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last"
        validation_split=0.0  # fraction of images reserved for validation (strictly between 0 and 1)
)

datagen.fit(x_train)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import ZeroPadding2D, Conv2D, MaxPooling2D, Dropout, Activation, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

import datetime
#NAME = 'cifar10-8blocks-elu-lr-schedule-l2-regulizer-horizontal-flipping-{}'.format(datetime.datetime.now())
NAME = 'cifar10-7blocks-my_activation-lr-schedule-l2-regulizer-horizontal-flipping-{}'.format(datetime.datetime.now())
tensorboard = TensorBoard(log_dir='./logs/{}'.format(NAME),
                          #histogram_freq=2,
                          #write_graph=True,
                          #write_grads=True,
                          #write_images=True,
                         )

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch, lr):
    if epoch < 70:
        return 0.01
    elif epoch < 170:
        return 0.005
    elif epoch < 270:
        return 0.0005
    else:
        return 0.00005
schedule = LearningRateScheduler(lr_schedule)

In [ ]:
import tensorflow as tf
def my_activation(x):
    return tf.where(x > 0.0,
                    tf.log(tf.maximum(x, 0.0) + 1.0),
                    -tf.log(-tf.minimum(x, 0.0) + 1.0))

In [ ]:
model = Sequential([
    #Block 1
    ZeroPadding2D(input_shape=(32, 32, 3)),
    Conv2D(384, 3, kernel_regularizer=l2(l=0.0005)), # 32
    Activation('relu'),
    MaxPooling2D(), # 16
    # Block 2
    Conv2D(384, 1, kernel_regularizer=l2(l=0.0005)), # 16
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(384, 2, kernel_regularizer=l2(l=0.0005)), # 17
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(640, 2, kernel_regularizer=l2(l=0.0005)), # 18
    Activation('relu'),
    MaxPooling2D(), # 9
    Dropout(0.1),
    # Block 3
    Conv2D(640, 1, kernel_regularizer=l2(l=0.0005)), # 9
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 10
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 11
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(768, 2, kernel_regularizer=l2(l=0.0005)), # 12
    Activation('relu'),
    MaxPooling2D(), # 6
    Dropout(0.2),
    # Block 4
    Conv2D(768, 1, kernel_regularizer=l2(l=0.0005)), # 6
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(896, 2, kernel_regularizer=l2(l=0.0005)), # 7
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(896, 2, kernel_regularizer=l2(l=0.0005)), # 8
    Activation('relu'),
    MaxPooling2D(), # 4
    Dropout(0.3),
    # Block 5
    Conv2D(896, 1, kernel_regularizer=l2(l=0.0005)), # 4
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(1024, 2, kernel_regularizer=l2(l=0.0005)), # 5
    Activation('relu'),
    ZeroPadding2D(),
    Conv2D(1024, 2, kernel_regularizer=l2(l=0.0005)), # 6
    Activation('relu'),
    MaxPooling2D(), # 3
    Dropout(0.4),
    # Block 6
    Conv2D(1024, 1, kernel_regularizer=l2(l=0.0005)), # 3
    Activation('relu'),
    Conv2D(1152, 2, kernel_regularizer=l2(l=0.0005)), # 2
    Activation('relu'),
    MaxPooling2D(), # 1
    Dropout(0.5),
    # Block 7
    Conv2D(1152, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('relu'),
    Dropout(0.5),
    # Block 8
    Conv2D(10, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('sigmoid'),
    Flatten(),
])

In [ ]:
model = Sequential([
    # Block 1
    ZeroPadding2D(padding=(4, 4), input_shape=(32, 32, 3)),
    Conv2D(192, 5, kernel_regularizer=l2(l=0.0005)), # 36
    Activation(my_activation),
    MaxPooling2D(), # 18
    # Block 2
    Conv2D(192, 1, kernel_regularizer=l2(l=0.0005)), # 18
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(240, 3, kernel_regularizer=l2(l=0.0005)), # 18
    Activation(my_activation),
    MaxPooling2D(), # 9
    Dropout(0.1),
    # Block 3
    Conv2D(240, 1, kernel_regularizer=l2(l=0.0005)), # 9
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(260, 2, kernel_regularizer=l2(l=0.0005)), # 10
    Activation(my_activation),
    MaxPooling2D(), # 5
    Dropout(0.2),
    # Block 4
    Conv2D(260, 1, kernel_regularizer=l2(l=0.0005)), # 5
    Activation(my_activation),
    ZeroPadding2D(),
    Conv2D(280, 2, kernel_regularizer=l2(l=0.0005)), # 6
    Activation(my_activation),
    MaxPooling2D(), # 3
    Dropout(0.3),
    # Block 5
    Conv2D(280, 1, kernel_regularizer=l2(l=0.0005)), # 3
    Activation(my_activation),
    Conv2D(300, 2), # 2
    Activation(my_activation),
    MaxPooling2D(), # 1
    Dropout(0.4),
    # Block 6
    Conv2D(300, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation(my_activation),
    Dropout(0.5),
    # Block 7
    Conv2D(10, 1, kernel_regularizer=l2(l=0.0005)), # 1
    Activation('sigmoid'),
    Flatten()
])

In [ ]:
from tensorflow.keras.metrics import top_k_categorical_accuracy

model.compile(#'adam',
              SGD(lr=0.01, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy', top_k_categorical_accuracy])

hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                           validation_data=[x_test, y_test],
                           epochs=330,
                           callbacks=[tensorboard, schedule]
                          )